In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,  LSTM, Reshape,  TimeDistributed
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
!pip install opendatasets

In [ ]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/dhruvsaluja/indian-medicinal-plants")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: dhruvsaluja
Your Kaggle Key: ··········


100%|██████████| 253M/253M [00:02<00:00, 99.9MB/s]


In [ ]:
# Define parameters
input_shape = (224, 224, 3)  # Adjust the image size as needed
batch_size = 32
epochs = 20
num_classes = 40  # Number of different plant classes
data_dir = "/content/indian-medicinal-plants/Indian Medicinal Plant Image Datasets/Medicinal plant dataset"  # Replace with the path to your dataset folder


In [ ]:
# Create a mapping between class indices and plant names
class_mapping = {
    0: 'Aloevera',
    1: 'Amla',
    2: 'Amruta Balli',
    3: 'Arali',
    4: 'Ashoka',
    5: 'Ashwagandha',
    6: 'Avacado',
    7: 'Bamboo',
    8: 'Basale',
    9: 'Betel',
    10: 'Betel_Nut',
    11: 'Brahmi',
    12: 'Castor',
    13: 'Curry Leaf',
    14: 'Doddapatre',
    15: 'Ekka',
    16: 'Ganike',
    17: 'Gauva',
    18: 'Geranium',
    19: 'Henna',
    20: 'Hibiscus',
    21: 'Honge',
    22: 'Insulin',
    23: 'Jasmine',
    24: 'Lemon',
    25: 'Lemon_grass',
    26: 'Mango',
    27: 'Mint',
    28: 'Nagadali',
    29: 'Neem',
    30: 'Nithyapushpa',
    31: 'Nooni',
    32: 'Pappaya',
    33: 'Pepper',
    34: 'Pomegranate',
    35: 'Raktachandini',
    36: 'Rose',
    37: 'Sapota',
    38: 'Tulasi',
    39: 'Wood_sorel',
    # Add mappings for all 40 classes here
}

In [ ]:
# Data preprocessing and augmentation
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,     # Normalize pixel values
    rotation_range=20,      # Randomly rotate images
    width_shift_range=0.2,  # Randomly shift images horizontally
    height_shift_range=0.2, # Randomly shift images vertically
    shear_range=0.2,        # Shear intensity
    zoom_range=0.2,         # Randomly zoom in on images
    horizontal_flip=True,   # Randomly flip images horizontally
    fill_mode='nearest',    # Fill missing pixels with the nearest value
    validation_split=0.2     # 20% of data will be used for validation
)

In [ ]:
# Load and augment training data (80%)
train_generator = datagen.flow_from_directory(
    data_dir,                 # Root directory containing 40 subfolders
    target_size=input_shape[:2],   # Resize images to match input_shape
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=True,             # Shuffle the data for training
    subset='training'         # Specify training data subset
)

Found 4765 images belonging to 40 classes.


In [ ]:
# Load and augment validation data (20%)
validation_generator = datagen.flow_from_directory(
    data_dir,                 # Root directory containing 40 subfolders
    target_size=input_shape[:2],  # Resize images to match input_shape
    batch_size=batch_size,
    class_mode='categorical',  # Categorical classification
    shuffle=False,            # Do not shuffle for validation
    subset='validation'       # Specify validation data subset
)


Found 1180 images belonging to 40 classes.


In [ ]:
# Load and augment training data (80%) for MobileNetV2
train_generator_mobilenet = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training',
    # classes=['class1', 'class2', ..., 'classN']  # List of class subfolders specific to MobileNetV2
)

# Load and augment validation data (20%) for MobileNetV2
validation_generator_mobilenet = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation',
    # classes=['class1', 'class2', ..., 'classN']  # List of class subfolders specific to MobileNetV2
)

# Repeat the same process for VGG16
# Load and augment training data (80%) for VGG16
train_generator_vgg = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training',
    # classes=['class1', 'class2', ..., 'classN']  # List of class subfolders specific to VGG16
)

# Load and augment validation data (20%) for VGG16
validation_generator_vgg = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation',
    # classes=['class1', 'class2', ..., 'classN']  # List of class subfolders specific to VGG16
)

# Repeat the same process for ResNet18
# Load and augment training data (80%) for ResNet18
train_generator_resnet = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training',
    # classes=['class1', 'class2', ..., 'classN']  # List of class subfolders specific to ResNet18
)

# Load and augment validation data (20%) for ResNet18
validation_generator_resnet = datagen.flow_from_directory(
    data_dir,
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    subset='validation',
    # classes=['class1', 'class2', ..., 'classN']  # List of class subfolders specific to ResNet18
)


Found 4765 images belonging to 40 classes.
Found 1180 images belonging to 40 classes.
Found 4765 images belonging to 40 classes.
Found 1180 images belonging to 40 classes.
Found 4765 images belonging to 40 classes.
Found 1180 images belonging to 40 classes.


In [ ]:
import numpy as np
from tensorflow.keras.utils import Sequence

class MultiDataGenerator(Sequence):
    def __init__(self, generators):
        self.generators = generators

    def __len__(self):
        return len(self.generators[0])

    def __getitem__(self, idx):
        batch_x = [generator[idx][0] for generator in self.generators]
        batch_y = self.generators[0][idx][1]  # Assuming the first generator provides both input and target
        return batch_x, batch_y


In [ ]:
from tensorflow.keras.layers import Input, Concatenate, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2, VGG16, ResNet50  # Import ResNet50 instead of ResNet18

from tensorflow.keras.metrics import Accuracy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# Define input shape
input_shape = (224, 224, 3)  # Assuming input images are 224x224 RGB images
num_classes = 40  # Define the number of classes in your dataset

# Load MobileNetV2
input_mobilenet = Input(shape=input_shape)
base_model_mobilenet = MobileNetV2(weights='imagenet', include_top=False, input_tensor=input_mobilenet)
for layer in base_model_mobilenet.layers:
    layer._name = layer.name + '_mobilenet'  # Append "_mobilenet" to each layer's name to make them unique
    layer.trainable = False
output_mobilenet = GlobalAveragePooling2D()(base_model_mobilenet.output)

# Load VGG16
input_vgg = Input(shape=input_shape)
base_model_vgg = VGG16(weights='imagenet', include_top=False, input_tensor=input_vgg)
for layer in base_model_vgg.layers:
    layer._name = layer.name + '_vgg'  # Append "_vgg" to each layer's name to make them unique
    layer.trainable = False
output_vgg = GlobalAveragePooling2D()(base_model_vgg.output)

# Load ResNet18
input_resnet = Input(shape=input_shape)
base_model_resnet = ResNet50(weights='imagenet', include_top=False, input_tensor=input_resnet)
for layer in base_model_resnet.layers:
    layer._name = layer.name + '_resnet'  # Append "_resnet" to each layer's name to make them unique
    layer.trainable = False
output_resnet = GlobalAveragePooling2D()(base_model_resnet.output)

# Concatenate outputs from all models
merged = Concatenate()([output_mobilenet, output_vgg, output_resnet])

# Add dense layers for classification
x = Dense(128, activation='relu')(merged)
predictions = Dense(num_classes, activation='softmax')(x)

# Create ensemble model
ensemble_model = Model(inputs=[input_mobilenet, input_vgg, input_resnet], outputs=predictions)

# Compile the ensemble model
ensemble_model.compile(optimizer=Adam(learning_rate=0.0001),  # Adjust the learning rate if needed
                       loss='categorical_crossentropy',
                       metrics=['accuracy', Accuracy()])

# Assuming you have defined train_generator_mobilenet, train_generator_vgg, train_generator_resnet,
# validation_generator_mobilenet, validation_generator_vgg, and validation_generator_resnet earlier

# Create a multi-data generator for training
train_multi_generator = MultiDataGenerator([train_generator_mobilenet, train_generator_vgg, train_generator_resnet])

# Create a multi-data generator for validation
validation_multi_generator = MultiDataGenerator([validation_generator_mobilenet, validation_generator_vgg, validation_generator_resnet])

# Train the model using the multi-data generator
history = ensemble_model.fit(
    train_multi_generator,
    epochs=1,
    validation_data=validation_multi_generator
)


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1249, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 592, in update_state
        self.build

In [ ]:

# Reset the validation generators
validation_generator_d121.reset()
validation_generator_d169.reset()
validation_generator_d201.reset()


In [ ]:

# Get the number of steps for validation data
validation_steps = len(validation_generator_d121)

# Initialize empty lists to store predictions and true labels
y_pred_probs_list = []
y_true_list = []

# Loop through validation generator batches to get predictions
for _ in range(validation_steps):
    # Get batches of validation data for each DenseNet model
    batch_x_d121, batch_y_d121 = validation_generator_d121.next()
    batch_x_d169, batch_y_d169 = validation_generator_d169.next()
    batch_x_d201, batch_y_d201 = validation_generator_d201.next()

    # Make predictions using the ensemble model for each batch
    y_pred_probs_batch = ensemble_model.predict([batch_x_d121, batch_x_d169, batch_x_d201])

    # Append predictions and true labels to the lists
    y_pred_probs_list.append(y_pred_probs_batch)
    y_true_list.append(batch_y_d121)  # Assuming all generators have the same true labels

# Concatenate predictions and true labels from all batches
y_pred_probs = np.concatenate(y_pred_probs_list)
y_true = np.concatenate(y_true_list)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_probs, axis=1)


# Convert one-hot encoded true labels to class labels
y_true_class = np.argmax(y_true, axis=1)

# Calculate evaluation metrics using class labels
accuracy = accuracy_score(y_true_class, y_pred)
precision = precision_score(y_true_class, y_pred, average='weighted')
recall = recall_score(y_true_class, y_pred, average='weighted')
f1 = f1_score(y_true_class, y_pred, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

# Display classification report and confusion matrix
class_names = list(class_mapping.values())
print("\nClassification Report:")
print(classification_report(y_true_class, y_pred, target_names=class_names))

print("\nConfusion Matrix:")
conf_matrix = confusion_matrix(y_true_class, y_pred)
print(conf_matrix)







1/1 [==============================] - 0s 170ms/step
Accuracy: 0.07288135593220339
Precision: 0.06858317226927753
Recall: 0.07288135593220339
F1 Score: 0.06070115846144688

Classification Report:
               precision    recall  f1-score   support

     Aloevera       0.16      0.47      0.24        32
         Amla       0.07      0.03      0.05        29
 Amruta Balli       0.00      0.00      0.00        29
        Arali       0.32      0.38      0.35        29
       Ashoka       0.00      0.00      0.00        29
  Ashwagandha       0.00      0.00      0.00        29
      Avacado       0.03      0.03      0.03        29
       Bamboo       0.06      0.03      0.04        29
       Basale       0.15      0.07      0.10        29
        Betel       0.14      0.17      0.15        30
    Betel_Nut       0.07      0.07      0.07        29
       Brahmi       0.00      0.00      0.00        29
       Castor       0.09      0.03      0.05        32
   Curry Leaf       0.14      0.1

In [ ]:

# Save the trained model
ensemble_model.save('ensemble_densenets.h5')

In [ ]:
ensemble_model.summary()

In [ ]:
# # Evaluate the model on the validation set
# validation_steps = len(validation_generator)
# validation_generator.reset()
# y_true = validation_generator.classes
# y_pred_probs = ensemble_model.predict(validation_generator, steps=validation_steps)
# y_pred = np.argmax(y_pred_probs, axis=1)

In [ ]:
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

# # Calculate evaluation metrics
# accuracy = accuracy_score(y_true, y_pred)
# precision = precision_score(y_true, y_pred, average='weighted')
# recall = recall_score(y_true, y_pred, average='weighted')
# f1 = f1_score(y_true, y_pred, average='weighted')

# print(f'Accuracy: {accuracy}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')
# print(f'F1 Score: {f1}')

# # Display classification report and confusion matrix
# class_names = list(class_mapping.values())
# print("\nClassification Report:")
# print(classification_report(y_true, y_pred, target_names=class_names))

# print("\nConfusion Matrix:")
# conf_matrix = confusion_matrix(y_true, y_pred)
# print(conf_matrix)

In [ ]:
# Save the trained model
# model.save('plant_classification_model_transfer.h5')

In [ ]:
# Load the model for inference
# loaded_model = keras.models.load_model('plant_classification_model_transfer.h5')


In [ ]:
# Example code for prediction on a new image
def predict_plant(image_path):
    img = image.load_img(image_path, target_size=input_shape[:2])
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img /= 255.0

    prediction = loaded_model.predict(img)
    class_index = np.argmax(prediction)
    plant_name = class_mapping.get(class_index, 'Unknown Plant')
    confidence = np.max(prediction)

    return plant_name, confidence

In [ ]:
# Example usage:
image_path = '/content/aloe.jpg'
predicted_plant, confidence = predict_plant(image_path)
print(f"Predicted plant: {predicted_plant}")
print(f"Confidence level: {confidence * 100:.2f}%")